## Electronic Components Classification 
In this project we make a model which classify the electronic components.

In [1]:
# importing libraries
import numpy as np
import pandas as pd
import os
import glob

In [2]:
# set the train and test path 
train_path = '/content/drive/MyDrive/ML Projects/Electronic Component Classification/data/train_data'
test_path = '/content/drive/MyDrive/ML Projects/Electronic Component Classification/data/test_data'

In [3]:
# list of component folder
categories = os.listdir(train_path)

In [4]:
categories

['Crystals',
 'Inductor',
 'Ceramic Capacitor',
 'Cartridge Fuse',
 'Clip Lead',
 'Shunt',
 'Armature',
 'Motor',
 'Microprocessor',
 'Automotive Fuse',
 'Mosfet',
 'Resistor',
 'Transformer',
 'Rheostat',
 'LED',
 'Battery',
 'Switch',
 'Electrolytic Capacitor',
 'Potentiometer',
 'Heat Sink',
 'Diode',
 'Transistor']

In [5]:
# sorting the categories folder
categories.sort()

In [6]:
# using the image data generator to make the image dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [7]:
# fixing the image size
image_size = [224,224]

In [8]:
# training image data generator
train_img_gen = ImageDataGenerator(rescale= 1/255, shear_range= 0.5, zoom_range= [0.5,1], 
                            horizontal_flip= True, vertical_flip = True, rotation_range=90,
                            fill_mode = 'nearest')

In [9]:
# test image data generator
test_img_gen = ImageDataGenerator(rescale = 1/255)

In [10]:
# creating the traning image data
train_data = train_img_gen.flow_from_directory(train_path, target_size = (224,224) , batch_size = 16, class_mode= 'categorical', 
                                         classes = categories)

Found 139 images belonging to 22 classes.


In [11]:
# creating the test image data
test_data = test_img_gen.flow_from_directory(test_path, target_size = (224,224) , batch_size = 16, class_mode= 'categorical', 
                                         classes = categories)

Found 62 images belonging to 22 classes.


In [12]:
train_data.image_shape

(224, 224, 3)

In [13]:
test_data.image_shape

(224, 224, 3)

## Creating the Model

In [15]:
# import basic libraries for model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D

### MobileNet Model

In [ ]:
# importing mobilenet model
from tensorflow.keras.applications.mobilenet import MobileNet

In [ ]:
# initializing mobilenet model
mobilenet = MobileNet(input_shape= image_size + [3] , weights = 'imagenet', include_top= False)

# we don't have to train the existing weights
for layer in mobilenet.layers:
  layer.trainable = False

# globalaveragepooling2d layer
mblnt_gap2d_layer = GlobalAveragePooling2D()(mobilenet.output)

# making dense layer
mblnt_dense_layer = Dense(units = len(categories), activation= 'softmax')(gap2d_layer)

# model object
mobilenet_model = Model(inputs = mobilenet.input, outputs = mblnt_dense_layer )

# compiling the model
mobilenet_model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )

In [ ]:
# model summary
mobilenet_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [ ]:
# fitting the model
mobilenet_result = mobilenet_model.fit_generator(train_data, validation_data= test_data, epochs = 30)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/30
9/9 [==============================] - 5s 396ms/step - loss: 3.3756 - accuracy: 0.0791 - val_loss: 3.0224 - val_accuracy: 0.1129
Epoch 2/30
9/9 [==============================] - 3s 317ms/step - loss: 2.3769 - accuracy: 0.2950 - val_loss: 2.4577 - val_accuracy: 0.3710
Epoch 3/30
9/9 [==============================] - 3s 313ms/step - loss: 1.7884 - accuracy: 0.5036 - val_loss: 1.9846 - val_accuracy: 0.4516
Epoch 4/30
9/9 [==============================] - 3s 313ms/step - loss: 1.3187 - accuracy: 0.6835 - val_loss: 1.7014 - val_accuracy: 0.5323
Epoch 5/30
9/9 [==============================] - 3s 314ms/step - loss: 1.0723 - accuracy: 0.7554 - val_loss: 1.4495 - val_accuracy: 0.5806
Epoch 6/30
9/9 [==============================] - 3s 323ms/step - loss: 0.8184 - accuracy: 0.8345 - val_loss: 1.3080 - val_accuracy: 0.5968
Epoch 7/30
9/9 [==============================] - 3s 309ms/step - loss: 0.6864 - accuracy: 0.8849 - val_loss: 1.2022 - val_accuracy: 0.5968
Epoch 8/30
9/9 [====

#### Accuracy of mobilenet model is 82%.

In [ ]:
model_folder = "/content/drive/MyDrive/ML Projects/Electronic Component Classification"
model_name = 'mobilenet_model_82acc_30ep.h5'
model_path = model_folder + '/' + model_name

In [ ]:
# saving the model
mobilenet_model.save(model_path)

### InceptionV3 Model

In [ ]:
# importing inceptionv3
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
# initializing inception model
inception = InceptionV3(input_shape= image_size + [3] , weights = 'imagenet', include_top= False)
# we don't have to train the existing weights
for layer in inception.layers:
  layer.trainable = False
# # flattening the inception layer
# inv3_flat_layer = Flatten()(inception.output)

# globalaveragepooling2d layer
inv3_gap2d_layer = GlobalAveragePooling2D()(inception.output)

# final dense layer
inv3_dense_layer = Dense(units = len(categories), activation= 'softmax')(inv3_gap2d_layer)

# model object
inceptionv3_model = Model(inputs = inception.input, outputs = inv3_dense_layer )

# compiling the model
inceptionv3_model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )

In [ ]:
# model summary
inceptionv3_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_188 (Conv2D)            (None, 111, 111, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_188 (Batch  (None, 111, 111, 32  96         ['conv2d_188[0][0]']             
 Normalization)                 )                                                           

In [ ]:
# fitting the model
inceptionv3_result = inceptionv3_model.fit_generator(train_data, validation_data= test_data, epochs = 40)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/40
9/9 [==============================] - 10s 569ms/step - loss: 3.6500 - accuracy: 0.0576 - val_loss: 2.9611 - val_accuracy: 0.1613
Epoch 2/40
9/9 [==============================] - 3s 357ms/step - loss: 2.5260 - accuracy: 0.2230 - val_loss: 2.2032 - val_accuracy: 0.4032
Epoch 3/40
9/9 [==============================] - 3s 358ms/step - loss: 1.6932 - accuracy: 0.5180 - val_loss: 1.6965 - val_accuracy: 0.5645
Epoch 4/40
9/9 [==============================] - 3s 357ms/step - loss: 1.2728 - accuracy: 0.6331 - val_loss: 1.3077 - val_accuracy: 0.6935
Epoch 5/40
9/9 [==============================] - 3s 355ms/step - loss: 1.0043 - accuracy: 0.7842 - val_loss: 1.1426 - val_accuracy: 0.7097
Epoch 6/40
9/9 [==============================] - 3s 348ms/step - loss: 0.7977 - accuracy: 0.8273 - val_loss: 0.9937 - val_accuracy: 0.7581
Epoch 7/40
9/9 [==============================] - 3s 354ms/step - loss: 0.6208 - accuracy: 0.8777 - val_loss: 0.9654 - val_accuracy: 0.7258
Epoch 8/40
9/9 [===

#### Inception model accuracy was around 92%.

In [ ]:
model_folder = "/content/drive/MyDrive/ML Projects/Electronic Component Classification"
inceptionv3_model_name = 'inceptionv3_model_92vacc_100tacc_40ep.h5'
inceptionv3_model_path = model_folder + '/' + inceptionv3_model_name

In [ ]:
# saving the inception3 model
inceptionv3_model.save(inceptionv3_model_path)

### Xception Model

In [16]:
# importing xception model
from tensorflow.keras.applications import Xception

In [17]:
# initializing the xception model
xception = Xception(input_shape = image_size + [3], weights = 'imagenet', include_top= False)
# not train the layers
for layer in xception.layers:
  layer.trainable = False

# # flat layer
# xcpt_flat_layer = Flatten()(xception.output)

# globalaveragpooling layer
xcept_gap2d_layer = GlobalAveragePooling2D()(xception.output)

# dense layer
xcept_dense_layer = Dense(units = len(categories), activation= 'softmax')(xcept_gap2d_layer)

# building model
xception_model = Model(inputs = xception.input , outputs = xcept_dense_layer)

# compiling model
xception_model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'] )

83697664/83683744 [==============================] - 1s 0us/step


In [18]:
# xception model summary
xception_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [19]:
# fitting xception model
xception_result = xception_model.fit_generator(train_data, validation_data= test_data, epochs = 30)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/30
9/9 [==============================] - 62s 6s/step - loss: 3.1180 - accuracy: 0.1151 - val_loss: 2.7729 - val_accuracy: 0.1774
Epoch 2/30
9/9 [==============================] - 4s 423ms/step - loss: 2.2874 - accuracy: 0.3453 - val_loss: 2.2042 - val_accuracy: 0.4032
Epoch 3/30
9/9 [==============================] - 4s 425ms/step - loss: 1.7407 - accuracy: 0.6691 - val_loss: 1.7465 - val_accuracy: 0.6290
Epoch 4/30
9/9 [==============================] - 4s 429ms/step - loss: 1.2833 - accuracy: 0.8058 - val_loss: 1.4376 - val_accuracy: 0.7097
Epoch 5/30
9/9 [==============================] - 4s 425ms/step - loss: 1.0225 - accuracy: 0.8705 - val_loss: 1.1959 - val_accuracy: 0.8226
Epoch 6/30
9/9 [==============================] - 4s 435ms/step - loss: 0.8029 - accuracy: 0.9209 - val_loss: 1.0401 - val_accuracy: 0.8387
Epoch 7/30
9/9 [==============================] - 4s 419ms/step - loss: 0.6728 - accuracy: 0.9496 - val_loss: 0.9356 - val_accuracy: 0.8548
Epoch 8/30
9/9 [======

#### Xception model accuracy was 92%.

In [20]:
model_folder = "/content/drive/MyDrive/ML Projects/Electronic Component Classification"
xception_model_name = 'xception_model_92vacc_98tacc_30ep.h5'
xception_model_path = model_folder + '/' + xception_model_name

In [21]:
xception.save(xception_model_path)

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


### Evaluation Functions

In [ ]:
import cv2
from skimage.transform import rescale, resize, downscale_local_mean

In [ ]:
def test_img(img_string):
  img = cv2.imread(img_string)
  resized_image = resize(img, (224,224))
  pred_img = np.expand_dims(resized_image, axis =0)
  return pred_img


In [ ]:
test_image = str(input())
pred_img = test_img(test_image)

### Keras Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
keras_model = load_model(keras_model_path)

In [ ]:
keras_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_5 (Sequential)   (None, 1280)              410208    
                                                                 
 sequential_7 (Sequential)   (None, 22)                130300    
                                                                 
Total params: 540,508
Trainable params: 526,428
Non-trainable params: 14,080
_________________________________________________________________


In [ ]:
import tensorflow as tf


In [ ]:
keras_y_pred = keras_model.predict(test_data)

In [ ]:
y_pred = []
for v in keras_y_pred:
  y_pred.append(v.argmax())

In [ ]:
keras_model.

### Tensorflow lite model


In [ ]:
import tensorflow as tf

In [ ]:
unq_model = tf.lite.Interpreter(model_path= unquantized_path)

In [ ]:
input_details = unq_model.get_input_details()

In [ ]:
output_details = unq_model.get_output_details()

In [ ]:
input_details[0]['index']

0

In [ ]:
new_img = pred_img.astype(dtype = 'float32')

In [ ]:
unq_model.allocate_tensors()

In [ ]:
unq_model.set_tensor(input_details[0]['index'],new_img)

In [ ]:
unq_model.invoke()

In [ ]:
ten_output = unq_model.get_tensor(output_details[0]['index'])

In [ ]:
ten_output.argmax()

12

In [ ]:
categories[ten_output.argmax()]

'Microprocessor'

In [ ]:
unq_model.reset_all_variables()

In [ ]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Load the model
model = load_model('keras_model.h5')

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1.
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
# Replace this with the path to your image
image = Image.open('<IMAGE_PATH>')
#resize the image to a 224x224 with the same strategy as in TM2:
#resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)
# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
print(prediction)


In [ ]:
img_pred = keras_model.predict(pred_img)

NameError: ignored

In [ ]:
# tensor model test
unq_model.set_tensor(unq_model[0])
ten_output = unq_model.get_tensor()

In [ ]:
img_class = img_pred.argmax()

In [ ]:
categories[img_class]

'Heat Sink'

In [ ]:
categories

In [ ]:
# plot the loss
import matplotlib.pyplot as plt
plt.plot(result.history['loss'], label='train loss')
plt.plot(result.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(result.history['accuracy'], label='train acc')
plt.plot(result.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save('/content/drive/MyDrive/ML Projects/Electronic Component Classification/inception_v3_model.h5')

In [ ]:
model = load_model('/content/drive/MyDrive/ML Projects/Electronic Component Classification/inception_v3_model.h5')

In [ ]:
categories_label = dict(zip(range(36),categories))

In [ ]:
train_data

In [ ]:
categories_label

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
img_path = '/content/drive/MyDrive/ML Projects/Electronic Component Classification/data/test_data/Microprocessor/microprocessor088.jpg'

In [ ]:
img = load_img(img_path, target_size= (224,224))

In [ ]:
arr_img = img_to_array(img)

In [ ]:
arr_img.shape

In [ ]:
new_arr_img = np.expand_dims(arr_img, axis =0)

In [ ]:
y = model.predict(new_arr_img)

In [ ]:
y_p = np.argmax(y, axis =1)
print(y_p)

In [ ]:
y_p = y_p[0]

In [ ]:
categories_label[y_p]